## Ensemble 1: Agregacion de modelos

En este post vamos a discutir una de las ideas mas potentes a la hora de enfrentarse a un problema de Machine learning, ya sea de clasificacion o de regresion, los ensembles.

Hasta bien entrado la decada de los 90 los modelos predictivos que se utilizaban eran modelos individuales, donde la decision para cada problema era encontrar el "learner" mas potente dado un problema planteado.

Con el paper Schapire (1990) se sentaron las bases para lo que se conoce como combinacion de modelos.

El termino tiene muchos sinonimos: Forecast aggregation, model averaging, mixture of experts, wisdom of the crowds o predictive combination, pero la base sobre lo que se asienta es la misma, mezclar modelos para generar una prediccion superior a cualquiera individual realizada por un unico modelo.

### 1 Linear ensembles.

Primero vamos a sentar la notacion para el post:

tenemos, los errores del modelo en cuestion: $error = f(x_i)-\hat{f(x_i)}$ donde $f(x_i)$ es el valor real de la funcion a predecir y $\hat{f(x_i)}$ es la estimacion de un modelo sobre los datos de entrada.

Para combinar dichos modelos asumimos $M$ modelos donde $\hat{y_{i,m}}$ es la predicion del modelo $m$ sobre el dato $x_i$, las predicciones se combinaran en una matriz $(I x M)$ llamada $E$, dicha combinacion lineal de la suma de los errores es igual a $Ew$ donde $w$ es un vector de dimension $(1xM)$ con los pesos asignados a cada modelo, con la restriccion: $\textbf{w}^T\textbf{1}_M=1$ donde $\textbf{1}_M$ es la funcion identidad que toma el valor de 1 cuanto M es verdadero y 0 en otro caso, es decir si el modelo ha sido agregado al ensemble.

Ahora lo que queremos es definir el cuadrado de los errores que en algebra entre vectores se representa por el producto entre dichos vectores y matrices (aparece duplicado).

Utilizando los multiplicadores de lagrange para la restriccion de que la suma acumulada de los pesos sumen 1 tenemos que $\mathcal{L}
(\textbf{w})=\textbf{w}^T\textbf{E}^T\textbf{E}\textbf{w}-\lambda (\textbf{w}^T\textbf{1}_M-1)$ igualando la derivada a 0 para hayar el minimo:
$$\frac{\partial}{\partial \textbf{w}}\mathcal{L}(\textbf{w})=\textbf{E}^T\textbf{E}\textbf{w}-\lambda \textbf{1}_M=0 \quad \Leftrightarrow \quad \textbf{w}=\lambda(\textbf{E}^T\textbf{E})^{-1}\textbf{1}_M,$$

Haciendo la derivada respecto de $\lambda$ tenemos que el optimo de los pesos es: $$\textbf{w}^*=\frac{(\textbf{E}^T\textbf{E})^{-1}\textbf{1}_M}{(\textbf{1}_M^T\textbf{E}^T\textbf{E})^{-1}\textbf{1}_M}$$

El problema aqui es que $\textbf{E}^T\textbf{E}$ es una matriz de covarianza de los errores, si hay correlacion entre los errores, es decir si muchos de los modelos estan haciendo las mismas predicciones, dicha matriz si la correlacion entre ambos modelos es alta el peso que se le dara al modelo de menor error sera mucho mayor que el que se le dara al de mayor error que tendra un peso grande pero negativo, esto puede provocar predicciones espurias que no queremos.

Vale tenemos varios modelos y queremos combinarlos porque creemos que la opinion de muchos cuenta mas que la opinion de uno solo, pero nos ha surgido el problema anteriormente citado, que podemos hacer para evitarlo?.

La respuesta reside en la diversificacion de los modelos utilizados.

### 2 Modelos de agregacion, bootstrap y bagging.

El abordaje de la diversificacion de los modelos con los que contamos empieza con una tecnica estadistica potente pero sencilla.

La tecnica en cuestion se llama bootstrap, la finalidad de este metodo es intentar acercarnos a propiedades de la poblacion (datos totales sobre un fenomeno) con nuestro dataset o sample de la poblacion.

En bootstrap se generan $B$ datasets de un tamano $N$ de nuestros datos, con reemplazamiento, que quiere decir que los  escogidos pueden volver a escogerse en futuros samples.

![bootstrap] (bootstrap.png)

Con estos datasets podemos calcular cantidades que nos fueran de ayuda para hacer hipotesis, intervalos de confianza, medias, medianas, desviaciones etc.

Ahora vamos a presentar el concepto de bagging.

Para que el bootstraping pueda sernos util a la hora de mejorar nuestras predicciones lo que tenemos que hacer es crear $M$ modelos sobre cada uno de los $B$ bootstrap datasets, estos modelos individuales harian una media de sus predicciones (en el caso de un problema de regresion) tal que:

$$\hat{f}_{bag}(x) = \frac{1}{B} \sum_{b=1}^{B} \hat{f}^b(x)$$

donde $\hat{f}^b(x)$ es el modelo que ajusta el dataset $b$

Para un problema de clasificacion lo que tenemos que hacer es:

$$\hat{f}_{bag}(x) = \underset{k}{\mathrm{argmax}} \hat{f}^b(x)$$

donde en este caso $\hat{f}^b(x)$ es una K-vector (dimension K)donde K es el numero de clases a predecir y cada indice $p_k(x)$ $\forall k: 1,...,K$ la proporcion de que los clasificadores individuales predigan la clase $k$ en x.

Otra forma de utilizar el bagging para la clasificacion es hacer una media de las probabilidades de que dicho x pertenezca a la clase k.